In [2]:
pip install -U spacy

Requirement already up-to-date: spacy in /home/evilmorty/anaconda3/lib/python3.7/site-packages (2.2.3)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
import numpy as np
import spacy
import random


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/evilmorty/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#using wordnet to generate synonyms
from nltk.corpus import wordnet
synonyms = []
antonyms = []

for syn in wordnet.synsets("realtor"):
    for l in syn.lemmas():
            synonyms.append(l.name())
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())
print(synonyms)

['Realtor']


In [3]:
#Examples of investor text being used to generate a corpus of investor text data
investor_text=[]
investor_text.append('My name is John. I am 34. I work in the information technology sector. I would like to invest $30000 in my RRSP this year. I do not like having military and tobacco in my portfolio. I care a lot about sustainable development.')
investor_text.append('My name is Greg. I am 44. I work in the information technology sector. I would like to invest $30000 in my RRSP this year. I do not like gambling. I care a lot about sustainable development.')
investor_text.append('My name is Craig. I am 51. I work in the information technology sector. I would like to invest $30000 this year. I do not like stocks with fossil fuels and I care a lot about sustainable development.')
investor_text.append('My name is Jen. I am 28. I work information technology sector. I would like to invest $30000 this year. I do not like stocks with fossil fuels and tobacco.')
investor_text.append('My name is Jenny. I am 39. I work in information technology sector and I do not like stocks with tobacco and I dont like gambling. I would like to invest $30000 this year. I do not like stocks with fossil fuels and military assets.')
investor_text.append('My name is Tom. I am 29. I work in the information technology sector and I hate alcohol. I would like to invest $30000 in my RRSP this year.')
investor_text.append('My name is Jordan. I am 35. I work information technology sector. I would like to invest $30000 in my RRSP this year. I do not like stocks with fossil fuels and tobacco.')
investor_text.append('My name is Tim. I am 29. I would like to invest $30000 in my RRSP this year. I work in the health care and I hate alcohol and tobacco')
investor_text.append('My name is Emma. I am 33. I work in the information technology sector and I do not like stocks with tobacco and I dont like gambling. I would like to invest $30000 this year.')
investor_text.append('My name is Novak. I work real estate and I do not like stocks with tobacco,gambling and military.I would like to invest $30000 this year.')

In [4]:
#Synonyms of flags
all_synonyms={}
all_synonyms["fossil fuels"]= ['gasoline', 'gasolene', 'gas', 'petrol','natural gas','petroleum', 'crude oil', 'crude', 'rock oil', 'fossil oil', 'oil','non-renewable energy','coal','bio fuels','diesel']
all_synonyms["military"]=['armed forces', 'armed services', 'military machine', 'war machine','troops','war','navy','defense']
all_synonyms["alcohol"]=['liquor','booze','drinking','hooch','alcoholic_drink', 'alcoholic_beverage', 'intoxicant', 'inebriant','wine','liqueur']
all_synonyms["tobacco"]=[ 'baccy','smoke', 'smoking','cigar','cigarette','weed','nicotine']
all_synonyms["gambling"]=[ 'gaming', 'play', 'gamble', 'chance', 'risk','take a chance','take chances','wager','betting','bets']
all_synonyms["information technology"]=['software','data science','IT','data processing','cyber security','computer engineer','infotech']
all_synonyms["health care"]=['medical','doctor','hospital','health services','medicaid','medicine']
all_synonyms["consumer staples"]=[]
all_synonyms["government"]=['government administration','govt','state','public services','bureaucracy','regime','big brother','politics','congress']
all_synonyms["real estate"]=['realtor','property']

In [5]:
#Programmatically adding sentences with different sectors
sectors = ["information technology","health care","government","consumer staples","real estate"]
for s in sectors:
    for sentence in investor_text:
            if sentence.find(s) != -1:
                for j in sectors:
                    if j!=s:
                        new_sentence = sentence.replace(s,j)
                        investor_text.append(new_sentence)
                        


In [6]:
split_text=[h.split('.') for h in investor_text]
all_text = []



In [7]:
flags = ["military","alcohol","tobacco","gambling","fossil fuels"] + sectors

In [8]:
print(len(split_text))
# split_text

558


In [10]:
#programmatically generating strings by replacing flags like military, tobacco, gambling, etc.
#by their synonyms in sentences
# sentences are labelled as the flag found in the sentence
data=[]
curset={}
for sentence in split_text:
    for line in sentence:
        curset[line] = []
        
print(len(curset))

for flag in flags :
    cur_list=[]
    for x in curset:
        cur_list.append(x)

    for cur_line in cur_list:
        if cur_line.find(flag) != -1:
            for j in all_synonyms[flag]:
                new_line = cur_line.replace(flag,j)
                if new_line not in curset:
                    curset[new_line]=[]
                    for t in curset[cur_line]:
                        curset[new_line].append(t)
                curset[new_line].append((j,flag))
            curset[cur_line].append((flag,flag))
    
# print(curset)

63


In [11]:

for data_line in curset:
    if len(curset[data_line])!=0 :
        flag_set=[]
        for cur_flag in curset[data_line]:
            if data_line.find(cur_flag[0] )!=-1:
                flag_set.append(cur_flag[1])
        data.append((data_line,flag_set[0]))

In [12]:
len(data)

31372

In [14]:
nlp = spacy.blank("en")
optimizer = nlp.begin_training()


In [15]:
# print(curset)

In [16]:
output_dir = "."


In [22]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

from spacy.util import minibatch, compounding


# @plac.annotations(
#     model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
#     output_dir=("Optional output directory", "option", "o", Path),
#     n_texts=("Number of texts to train from", "option", "t", int),
#     n_iter=("Number of training iterations", "option", "n", int),
#     init_tok2vec=("Pretrained tok2vec weights", "option", "t2v", Path),
# )
def main(model=None, output_dir=None, n_iter=20, n_texts=1000, init_tok2vec=None):
    output_dir = "."
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()

    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # add the text classifier to the pipeline if it doesn't exist
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "textcat" not in nlp.pipe_names:
        textcat = nlp.create_pipe(
            "textcat", config={"exclusive_classes": False, "architecture": "simple_cnn"}
        )
        nlp.add_pipe(textcat, last=True)
    # otherwise, get it, so we can add labels to it
    else:
        textcat = nlp.get_pipe("textcat")

    # add label to text classifier
    textcat.add_label("military")
    textcat.add_label("alcohol")
    textcat.add_label("tobacco")
    textcat.add_label("gambling")
    textcat.add_label("fossil fuels")
    textcat.add_label("information technology")
    textcat.add_label("health Care")
    textcat.add_label("government")
    textcat.add_label("consumer staples")
    textcat.add_label("real estate")

    # load the dataset
    print("Loading data...")
    (train_texts, train_cats), (dev_texts, dev_cats) = load_data()
    train_texts = train_texts[:n_texts]
    train_cats = train_cats[:n_texts]
    print(
        "Using {} examples ({} training, {} evaluation)".format(
            n_texts, len(train_texts), len(dev_texts)
        )
    )
    train_data = list(zip(train_texts, [{"cats": cats} for cats in train_cats]))
#     print (train_data)

    # get names of other pipes to disable them during training
    pipe_exceptions = ["textcat", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train textcat
        optimizer = nlp.begin_training()
        if init_tok2vec is not None:
            with init_tok2vec.open("rb") as file_:
                textcat.model.tok2vec.from_bytes(file_.read())
        print("Training the model...")
        print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
        batch_sizes = compounding(4.0, 32.0, 1.001)
        for i in range(n_iter):
            losses = {}
            # batch up the examples using spaCy's minibatch
            random.shuffle(train_data)
            batches = minibatch(train_data, size=batch_sizes)
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
            with textcat.model.use_params(optimizer.averages):
                # evaluate on the dev data split off in load_data()
                scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
            print(
                "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table
                    losses["textcat"],
                    scores["textcat_p"],
                    scores["textcat_r"],
                    scores["textcat_f"],
                )
            )

    # test the trained model
    test_text = "My name is Emma. I am 33. I work in the information technology sector and I do not like stocks with tobacco and I dont like gambling. I would like to invest $30000 this year."
    doc = nlp(test_text)
    print(test_text, doc.cats)

    if output_dir is not None:
        with nlp.use_params(optimizer.averages):
            nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        print(test_text, doc2.cats)


def load_data(limit=0, split=0.8):
    # Partition off part of the train data for evaluation
    train_data = data
    random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats=[]
    for line in labels:
        dis={}
        for i in flags :
            if i in line:
                  dis[i]=True
            else:
                dis[i]=False
        cats.append(dis)
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])


def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "NEGATIVE":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}


main()

Created blank 'en' model
Loading data...
Using 1000 examples (1000 training, 6275 evaluation)
Training the model...
LOSS 	  P  	  R  	  F  
1.324	0.992	0.994	0.993
0.048	0.993	0.993	0.993
0.016	0.993	0.993	0.993
0.013	0.992	0.994	0.993
0.007	0.992	0.995	0.993
0.004	0.993	0.995	0.994
0.003	0.994	0.994	0.994
0.002	0.994	0.994	0.994
0.001	0.994	0.994	0.994
0.001	0.994	0.994	0.994
0.001	0.994	0.994	0.994
0.001	0.994	0.995	0.994
0.001	0.994	0.994	0.994
0.000	0.994	0.994	0.994
0.000	0.994	0.994	0.994
0.000	0.994	0.994	0.994
0.000	0.994	0.994	0.994
0.000	0.994	0.994	0.994
0.000	0.994	0.994	0.994
0.000	0.994	0.994	0.994
My name is Emma. I am 33. I work in the information technology sector and I do not like stocks with tobacco and I dont like gambling. I would like to invest $30000 this year. {'military': 0.00033698396873660386, 'alcohol': 0.001311403582803905, 'tobacco': 0.9820465445518494, 'gambling': 4.539787187241018e-05, 'fossil fuels': 0.0001078532513929531, 'information technology': 4.53

In [27]:
test_text='I work in the information technology sector and I do not like stocks with tobacco and I dont like gambling'
output_dir = "."
nlp2 = spacy.load(output_dir)
doc2 = nlp2(test_text)
print(test_text, doc2.cats)

I work in the information technology sector and I do not like stocks with tobacco and I dont like gambling {'military': 0.00019675570365507156, 'alcohol': 0.0010964245302602649, 'tobacco': 0.9989244341850281, 'gambling': 4.539787187241018e-05, 'fossil fuels': 0.000180625676875934, 'information technology': 5.741332279285416e-05, 'health Care': 0.4999998211860657, 'government': 7.983249088283628e-05, 'consumer staples': 4.539787187241018e-05, 'real estate': 0.00010042887879535556}


In [19]:
textcat = nlp2.get_pipe("textcat")

In [44]:
textcat.predict([doc2])

(array([[4.5397872e-05, 1.1054074e-04, 9.9995458e-01, 4.7750810e-01,
         4.5397872e-05, 4.5397872e-05, 5.0000030e-01, 3.0892214e-04,
         4.5397872e-05, 4.9583956e-05]], dtype=float32),
 [array([], dtype=float32)])

In [45]:
print(scores)

[[1.9675570e-04 1.0964245e-03 9.9892443e-01 4.5397872e-05 1.8062568e-04
  5.7413323e-05 4.9999982e-01 7.9832491e-05 4.5397872e-05 1.0042888e-04]]


In [39]:
print(tensors)

[array([], dtype=float32)]


In [21]:
test_text = "I like cigarette stocks"
doc2 = nlp2(test_text)
print(doc2.cats)
textcat.predict([doc2])

{'military': 0.0016351987142115831, 'alcohol': 4.539787187241018e-05, 'tobacco': 0.9998078942298889, 'gambling': 0.0005756979808211327, 'fossil fuels': 4.539787187241018e-05, 'information technology': 4.539787187241018e-05, 'health Care': 0.5000003576278687, 'government': 4.539787187241018e-05, 'consumer staples': 4.539787187241018e-05, 'real estate': 4.652397183235735e-05}


(array([[1.6351987e-03, 4.5397872e-05, 9.9980789e-01, 5.7569798e-04,
         4.5397872e-05, 4.5397872e-05, 5.0000036e-01, 4.5397872e-05,
         4.5397872e-05, 4.6523972e-05]], dtype=float32),
 [array([], dtype=float32)])